In [22]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Step 1: Load and preprocess datasets
def load_datasets():
    try:
        # Define file paths for the datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'
        
        # Load datasets into Pandas DataFrames
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)
        
        # Standardize column names: remove units in parentheses and convert to lowercase
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
        
        # Convert all string values in the dataframes to lowercase
        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        
        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Verify datasets are loaded successfully
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    print("All datasets are ready to use!")
else:
    print("Please check the dataset file paths and ensure the files exist.")

# Step 2: Main process function
def main_process(crops_df, nutrients_df, age_nutrients_df):
    # User Input: Get the district name
    district_name = input("Enter the district name: ").strip().lower()
    
    if district_name not in crops_df['district_name'].str.lower().unique():
        print(f"District '{district_name}' not found in the dataset. Please check and try again.")
        return None
    
    print(f"District '{district_name}' found in the dataset!")
    
    # Step 3: Identify Nutrient Deficiency
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None
    
    print(f"Crop data for district '{district_name}' loaded successfully.")
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c', 
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)
    
    # Compare with RDI for each age group
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict
    
    for age_group, deficiency in deficiencies.items():
        print(f"\nNutrient deficiencies for '{age_group}':")
        if deficiency:
            for nutrient, amount in deficiency.items():
                print(f"- {nutrient}: {amount:.2f}")
        else:
            print("No deficiencies.")
    
    # Step 4: Project Future Deficiencies
    future_projections = {}
    district_nutrient_data['year'] = district_nutrient_data['crop_year']
    yearly_nutrient_totals = district_nutrient_data.groupby('year')[nutrient_columns].sum().reset_index()
    
    for age_group, deficiency in deficiencies.items():
        future_projections[age_group] = {}
        print(f"\nProjected deficiencies for {age_group}:")
        for nutrient, current_deficiency in deficiency.items():
            if nutrient not in yearly_nutrient_totals.columns:
                print(f"Insufficient data for projecting {nutrient}.")
                continue
            X = yearly_nutrient_totals['year'].values.reshape(-1, 1)
            y = yearly_nutrient_totals[nutrient].values
            if len(X) < 2:
                print(f"Not enough historical data to project {nutrient}.")
                continue
            model = LinearRegression()
            model.fit(X, y)
            future_years_range = np.arange(X.max() + 1, X.max() + 1 + 10).reshape(-1, 1)
            future_deficiencies = model.predict(future_years_range)
            future_projections[age_group][nutrient] = current_deficiency + future_deficiencies[-1]
            print(f"- {nutrient}: {future_deficiencies[-1]:.2f}")
    
    # Step 5: Suggest Crops to Balance Nutrients
    recommended_crops = {}
    for age_group, deficiency in deficiencies.items():
        recommended_crops[age_group] = {}
        for nutrient, deficit_value in deficiency.items():
            nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
            recommended_crops[age_group][nutrient] = nutrient_rich_crops['crop'].tolist()
            print(f"\nRecommended crops for {nutrient} deficiency ({age_group}): {nutrient_rich_crops['crop'].tolist()}")
    
    # Step 6: Search Neighboring Districts for Surplus
    state_name = selected_district_crops['state_name'].iloc[0]
    neighboring_districts = crops_df[crops_df['state_name'] == state_name]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)
    print(f"\nNeighboring districts: {neighboring_districts}")
    
    crop_exchanges = {}
    for age_group, crops in recommended_crops.items():
        crop_exchanges[age_group] = {}
        for nutrient, crop_list in crops.items():
            for crop in crop_list:
                for neighbor in neighboring_districts:
                    neighbor_data = crops_df[crops_df['district_name'] == neighbor].merge(
                        nutrients_df, on='crop', how='inner'
                    )
                    nutrient_total = neighbor_data[nutrient].sum()
                    if nutrient_total > deficiencies[age_group][nutrient] * 1.2:  # Threshold surplus
                        crop_exchanges[age_group][nutrient] = {'donor_district': neighbor, 'available_amount': nutrient_total}
    
    print("\nCrop Exchange Suggestions:")
    print(crop_exchanges)
    return crop_exchanges

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Run the full process if datasets are successfully loaded
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    crop_exchanges = main_process(crops_df, nutrients_df, age_nutrients_df)


/tmp/ipykernel_353409/930819119.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_353409/930819119.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_353409/930819119.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_353409/930819119.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!
All datasets are ready to use!


/tmp/ipykernel_353409/930819119.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_353409/930819119.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!


KeyboardInterrupt: Interrupted by user

In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def calculate_high_deficiency(district_name, crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name.lower()]
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None

    print(f"Crop data for district '{district_name}' loaded successfully.")

    # Step 2: Merge with nutrient dataset and calculate total available nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 3: Compare available nutrients with RDI for each group
    deficiencies = {}
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Step 4: Identify high deficiency groups
    high_deficiency_groups = {group: def_dict for group, def_dict in deficiencies.items() if def_dict}
    print("\nGroups with High Nutrient Deficiencies:")
    for group, def_dict in high_deficiency_groups.items():
        print(f"- {group}:")
        for nutrient, amount in def_dict.items():
            print(f"  * {nutrient}: {amount:.2f}")
    
    # Step 5: Suggest crops for deficient nutrients
    crop_recommendations = {}
    for group, def_dict in high_deficiency_groups.items():
        crop_recommendations[group] = {}
        for nutrient, deficit in def_dict.items():
            nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
            crop_recommendations[group][nutrient] = nutrient_rich_crops[['crop', nutrient]].to_dict(orient='records')
    
    print("\nRecommended Crops for Deficient Nutrients:")
    for group, rec_dict in crop_recommendations.items():
        print(f"- {group}:")
        for nutrient, crops in rec_dict.items():
            print(f"  * {nutrient}:")
            for crop in crops[:3]:  # Limit to top 3 crops
                print(f"    - {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    
    # Step 6: Predict crop production over the next 5 years
    production_predictions = {}
    for group, rec_dict in crop_recommendations.items():
        production_predictions[group] = {}
        for nutrient, crops in rec_dict.items():
            for crop in crops[:3]:  # Limit to top 3 crops
                crop_name = crop['crop']
                crop_data = selected_district_crops[selected_district_crops['crop'].str.lower() == crop_name.lower()]
                if crop_data.empty:
                    continue

                yearly_production = crop_data.groupby('crop_year')['production'].sum().reset_index()
                if len(yearly_production) < 2:
                    print(f"Not enough data to predict production for {crop_name}.")
                    continue

                X = yearly_production['crop_year'].values.reshape(-1, 1)
                y = yearly_production['production'].values

                model = LinearRegression()
                model.fit(X, y)

                future_years = np.arange(X.max() + 1, X.max() + 6).reshape(-1, 1)
                future_production = model.predict(future_years)
                production_predictions[group].setdefault(nutrient, {})[crop_name] = {
                    'future_production': future_production.tolist(),
                    'required_deficit': deficit
                }
    
    print("\nPredicted Production for Recommended Crops:")
    for group, nutrients in production_predictions.items():
        print(f"- {group}:")
        for nutrient, crops in nutrients.items():
            print(f"  * {nutrient}:")
            for crop, details in crops.items():
                print(f"    - {crop}: Future Production: {details['future_production']}, Deficit to Fulfill: {details['required_deficit']:.2f}")
    
    return high_deficiency_groups, crop_recommendations, production_predictions

# Example Usage
district_name = "chitradurga"  # Replace with dynamic user input if required
high_deficiency_groups, crop_recommendations, production_predictions = calculate_high_deficiency(
    district_name, crops_df, nutrients_df, age_nutrients_df)


Crop data for district 'chitradurga' loaded successfully.
Total nutrients available in district 'chitradurga':
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Groups with High Nutrient Deficiencies:
- adult male crossbred (1000 number):
  * carbohydrates: 109470305.00
  * protein: 20431825.70
  * fiber: 10945966.10
  * fat: 25540077.40
  * vitamin a: 328398766.00
  * vitamin c: 32845918.80
  * iron: 2917472.60
  * calcium: 364999873.80
  * potassium: 1715187275.00
  * magnesium: 145913391.00
- adult female crossbred (1000 number):
  * carbohydrates: 109470305.00
  * protein: 16831825.70
  * fiber: 9745966.10
  * fat: 25540077.40
  * vitamin a: 255398766.00
  * vitamin c: 27370918.80
  * iron: 6567472.60
  * calcium: 364999873.80
  * potassium: 1715187275.00
  * 

Not enough data to predict production for papaya.
Not enough data to predict production for papaya.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
Not enough data to predict production for cashewnut processed.
Not enough data to predict production for soyabean.
Not enough data to predict production for papaya.
Not enough data to predict production for papaya.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
Not enough data to predict production for cashewnut processed.
Not enough data to predict production for soyabean.
Not enough data to predict production for papaya.
Not enough data to predict production for papaya.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
Not enough data to predict production for soyabean.
No

In [12]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def calculate_targeted_deficiency(district_name, crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name.lower()]
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None

    print(f"Crop data for district '{district_name}' loaded successfully.")

    # Step 2: Merge with nutrient dataset and calculate total available nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 3: Compare available nutrients with RDI for each group
    deficiencies = {}
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Step 4: Identify the most deficient groups
    high_deficiency_groups = {}
    for group, def_dict in deficiencies.items():
        if def_dict:  # Only add groups with deficiencies
            total_deficiency = sum(def_dict.values())
            high_deficiency_groups[group] = {'deficiency': def_dict, 'total_deficiency': total_deficiency}
    
    # Sort by total deficiency
    sorted_groups = sorted(high_deficiency_groups.items(), key=lambda x: x[1]['total_deficiency'], reverse=True)
    if not sorted_groups:
        print("No significant deficiencies found.")
        return None
    
    # Focus only on the top deficient group
    most_deficient_group, deficiency_data = sorted_groups[0]
    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiency_data['deficiency'].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Step 5: Recommend crops for deficient nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiency_data['deficiency'].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    
    # Step 6: Predict production for suggested crops
    print("\nProduction Prediction for Suggested Crops (Next 5 Years):")
    production_predictions = {}
    for nutrient, crops in recommended_crops.items():
        for crop in crops:
            crop_name = crop['crop']
            crop_data = selected_district_crops[selected_district_crops['crop'].str.lower() == crop_name.lower()]
            if crop_data.empty:
                continue

            yearly_production = crop_data.groupby('crop_year')['production'].sum().reset_index()
            if len(yearly_production) < 2:
                print(f"Not enough data to predict production for {crop_name}.")
                continue

            X = yearly_production['crop_year'].values.reshape(-1, 1)
            y = yearly_production['production'].values

            model = LinearRegression()
            model.fit(X, y)

            future_years = np.arange(X.max() + 1, X.max() + 6).reshape(-1, 1)
            future_production = model.predict(future_years)
            production_predictions[crop_name] = {
                'future_production': future_production.tolist(),
                'required_deficit': deficiency_data['deficiency'][nutrient]
            }
            print(f"- {crop_name}:")
            print(f"  Future Production: {future_production.tolist()}")
            print(f"  Deficit to Fulfill: {deficiency_data['deficiency'][nutrient]:.2f}")
    
    return most_deficient_group, recommended_crops, production_predictions

# Example Usage
district_name = "chitradurga"  # Replace with dynamic user input if required
most_deficient_group, recommended_crops, production_predictions = calculate_targeted_deficiency(
    district_name, crops_df, nutrients_df, age_nutrients_df)


Crop data for district 'chitradurga' loaded successfully.
Total nutrients available in district 'chitradurga':
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109470305.00
- protein: 21031825.70
- fiber: 12495966.10
- fat: 25490077.40
- vitamin a: 328398766.00
- vitamin c: 29245918.80
- iron: 4397472.60
- calcium: 474499873.80
- potassium: 1715187275.00
- magnesium: 148163391.00

Suggested Crops for Deficient Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0)
  * groundnut (Nutrient Value: 25.8)


In [15]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def nutrient_exchange_policy(district_name, crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name.lower()]
    if selected_district_crops.empty:
        print(f"No crop data found for the district: {district_name}")
        return None

    print(f"Crop data for district '{district_name}' loaded successfully.")
    
    # Step 2: Merge crop data with nutrient dataset and calculate total nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 3: Compare available nutrients with RDI to calculate deficiencies
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify most deficient group
    most_deficient_group = max(deficiencies, key=lambda g: sum(deficiencies[g].values()), default=None)
    if not most_deficient_group:
        print("No significant deficiencies found.")
        return None
    
    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiencies[most_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Step 4: Recommend crops for the deficient nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiencies[most_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # Step 5: Identify neighboring districts with surplus crops
    print("\nNeighboring Districts with Surplus Crops for Exchange:")
    surplus_crops = {}
    neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)
    for nutrient in deficiencies[most_deficient_group]:
        surplus_crops[nutrient] = []
        for neighbor in neighboring_districts:
            neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
            neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
            neighbor_surplus = neighbor_data[nutrient].sum() * 0.3  # Check if surplus is 30% of the nutrient
            if neighbor_surplus >= deficiencies[most_deficient_group][nutrient]:
                surplus_crops[nutrient].append({
                    'district': neighbor,
                    'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                    'surplus': neighbor_surplus
                })
    for nutrient, districts in surplus_crops.items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")

    # Step 6: Draft exchange policy
    print("\nExchange Policy for Nutrient Balance:")
    exchange_policy = {}
    for nutrient, districts in surplus_crops.items():
        if not districts:
            print(f"No surplus available for {nutrient}.")
            continue
        exchange_policy[nutrient] = []
        for district in districts:
            exchange_policy[nutrient].append({
                'receiving_district': district_name,
                'donor_district': district['district'],
                'crops_exchanged': district['crops'],
                'deficit_fulfilled': deficiencies[most_deficient_group][nutrient],
                'remaining_surplus': district['surplus'] - deficiencies[most_deficient_group][nutrient]
            })
    for nutrient, exchanges in exchange_policy.items():
        print(f"- {nutrient}:")
        for exchange in exchanges:
            print(f"  Receiving: {exchange['receiving_district']}, Donor: {exchange['donor_district']}")
            print(f"  Crops Exchanged: {exchange['crops_exchanged']}")
            print(f"  Deficit Fulfilled: {exchange['deficit_fulfilled']}")
            print(f"  Remaining Surplus: {exchange['remaining_surplus']}")

    return {
        "most_deficient_group": most_deficient_group,
        "deficiencies": deficiencies[most_deficient_group],
        "recommended_crops": recommended_crops,
        "surplus_crops": surplus_crops,
        "exchange_policy": exchange_policy
    }

# Example Usage
district_name = "chitradurga"
results = nutrient_exchange_policy(district_name, crops_df, nutrients_df, age_nutrients_df)

# Structured Output Example
print("\n=== Output Summary ===")
print(f"Nutrient Deficiency for District: {district_name}")
print(f"Most Deficient Group: {results['most_deficient_group']}")
print("Deficient Nutrients:")
for nutrient, amount in results['deficiencies'].items():
    print(f"- {nutrient}: {amount:.2f}")
print("\nSuggested Crops for Deficient Nutrients:")
for nutrient, crops in results['recommended_crops'].items():
    print(f"- {nutrient}:")
    for crop in crops:
        print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
print("\nExchange Policy for Nutrient Balance:")
for nutrient, exchanges in results['exchange_policy'].items():
    print(f"- {nutrient}:")
    for exchange in exchanges:
        print(f"  Receiving: {exchange['receiving_district']}, Donor: {exchange['donor_district']}")
        print(f"  Crops Exchanged: {exchange['crops_exchanged']}")
        print(f"  Deficit Fulfilled: {exchange['deficit_fulfilled']}")
        print(f"  Remaining Surplus: {exchange['remaining_surplus']}")


Crop data for district 'chitradurga' loaded successfully.
Total nutrients available in district 'chitradurga':
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109470305.00
- protein: 21031825.70
- fiber: 12495966.10
- fat: 25490077.40
- vitamin a: 328398766.00
- vitamin c: 29245918.80
- iron: 4397472.60
- calcium: 474499873.80
- potassium: 1715187275.00
- magnesium: 148163391.00

Suggested Crops for Deficient Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0)
  * groundnut (Nutrient Value: 25.8)


In [18]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def nutrient_exchange_analysis(crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Accept User Input for District Name
    district_name = input("Enter the district name: ").strip().lower()  # Convert input to lowercase
    
    # Validate the district name
    if district_name not in crops_df['district_name'].str.lower().unique():
        print(f"District '{district_name}' not found in the dataset. Please check the input.")
        return None

    print(f"\nDistrict '{district_name}' found in the dataset. Proceeding with analysis...\n")

    # Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
    
    # Merge crop data with nutrient dataset to calculate total nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Compare available nutrients with RDI to calculate deficiencies
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
    deficiencies = {}
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify the most deficient group
    most_deficient_group = max(deficiencies, key=lambda g: sum(deficiencies[g].values()), default=None)
    if not most_deficient_group:
        print("No significant deficiencies found.")
        return None

    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiencies[most_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Recommend crops for the deficient nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiencies[most_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # Analyze neighboring districts for surplus crops
    print("\nNeighboring Districts with Surplus Crops:")
    surplus_crops = {}
    neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)
    for nutrient in deficiencies[most_deficient_group]:
        surplus_crops[nutrient] = []
        for neighbor in neighboring_districts:
            neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
            neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
            neighbor_surplus = neighbor_data[nutrient].sum() * 0.3  # Check if surplus is 30% of the nutrient
            if neighbor_surplus >= deficiencies[most_deficient_group][nutrient]:
                surplus_crops[nutrient].append({
                    'district': neighbor,
                    'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                    'surplus': neighbor_surplus
                })
    for nutrient, districts in surplus_crops.items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")

    return {
        "most_deficient_group": most_deficient_group,
        "deficiencies": deficiencies[most_deficient_group],
        "recommended_crops": recommended_crops,
        "surplus_crops": surplus_crops
    }

# Example Usage
results = nutrient_exchange_analysis(crops_df, nutrients_df, age_nutrients_df)

# Structured Output
if results:
    print("\n=== Output Summary ===")
    print(f"Most Deficient Group: {results['most_deficient_group']}")
    print("Deficient Nutrients:")
    for nutrient, amount in results['deficiencies'].items():
        print(f"- {nutrient}: {amount:.2f}")
    print("\nSuggested Crops for Deficient Nutrients:")
    for nutrient, crops in results['recommended_crops'].items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    print("\nNeighboring Districts with Surplus Crops:")
    for nutrient, districts in results['surplus_crops'].items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")


Enter the district name: tumkur

District 'tumkur' found in the dataset. Proceeding with analysis...

Total nutrients available in district 'tumkur':
carbohydrates     32078.5
protein            8789.9
fiber              4370.9
fat                9916.8
vitamin a        313905.0
vitamin c          5240.0
iron               2784.9
calcium             140.8
potassium        343926.0
magnesium         94072.0
dtype: float64

Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109467921.50
- protein: 21031210.10
- fiber: 12495629.10
- fat: 25490083.20
- vitamin a: 328186095.00
- vitamin c: 29244760.00
- iron: 4397215.10
- calcium: 474499859.20
- potassium: 1715156074.00
- magnesium: 148155928.00

Suggested Crops for Deficient Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def calculate_targeted_deficiency(crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Dynamic User Input for District Name
    district_name = input("Enter the district name: ").strip().lower()  # Normalize input for comparison
    
    # Validate the district name
    if district_name not in crops_df['district_name'].str.lower().unique():
        print(f"District '{district_name}' not found in the dataset. Please check the input.")
        return None

    print(f"\nDistrict '{district_name}' found in the dataset. Proceeding with analysis...\n")

    # Step 2: Filter Crop Data for the Selected District
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
    if selected_district_crops.empty:
        print(f"No crop data available for district '{district_name}'.")
        return None

    # Merge with Nutrient Dataset and Calculate Total Available Nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c', 
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 3: Calculate Nutrient Deficiencies for Each Group
    deficiencies = {}
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify the Most Deficient Group
    high_deficiency_groups = {group: sum(def_dict.values()) for group, def_dict in deficiencies.items() if def_dict}
    if not high_deficiency_groups:
        print("No significant deficiencies found.")
        return None

    most_deficient_group = max(high_deficiency_groups, key=high_deficiency_groups.get)
    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiencies[most_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Step 4: Recommend Crops for Deficient Nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiencies[most_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # Step 5: Predict Future Production for Suggested Crops
    print("\nProduction Prediction for Suggested Crops (Next 5 Years):")
    production_predictions = {}
    for nutrient, crops in recommended_crops.items():
        for crop in crops:
            crop_name = crop['crop']
            crop_data = selected_district_crops[selected_district_crops['crop'].str.lower() == crop_name.lower()]
            if crop_data.empty:
                continue

            yearly_production = crop_data.groupby('crop_year')['production'].sum().reset_index()
            if len(yearly_production) < 2:
                print(f"Not enough data to predict production for {crop_name}.")
                continue

            # Predict Future Production Using Linear Regression
            X = yearly_production['crop_year'].values.reshape(-1, 1)
            y = yearly_production['production'].values
            model = LinearRegression()
            model.fit(X, y)

            future_years = np.arange(X.max() + 1, X.max() + 6).reshape(-1, 1)
            future_production = model.predict(future_years)
            production_predictions[crop_name] = {
                'future_production': future_production.tolist(),
                'required_deficit': deficiencies[most_deficient_group][nutrient]
            }
            print(f"- {crop_name}: Future Production: {future_production.tolist()}")
            print(f"  Deficit to Fulfill: {deficiencies[most_deficient_group][nutrient]:.2f}")

    return {
        "most_deficient_group": most_deficient_group,
        "recommended_crops": recommended_crops,
        "production_predictions": production_predictions
    }

# Example Usage
results = calculate_targeted_deficiency(crops_df, nutrients_df, age_nutrients_df)

# Output Results
if results:
    print("\n=== Output Summary ===")
    print(f"Most Deficient Group: {results['most_deficient_group']}")
    print("\nSuggested Crops for Deficient Nutrients:")
    for nutrient, crops in results['recommended_crops'].items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    print("\nProduction Predictions:")
    for crop, prediction in results['production_predictions'].items():
        print(f"- {crop}: Future Production: {prediction['future_production']}, Deficit: {prediction['required_deficit']:.2f}")
